In [ ]:
import os
import json
import zipfile
import copy
import re
from tqdm import tqdm
from rich import print as print_rich
import warnings

# ===========================
# Отключение предупреждений
# ===========================
warnings.filterwarnings("ignore", category=SyntaxWarning)

# ===========================
# Пути и константы
# ===========================
BASE_DIR = "/kaggle"
SUBMISSION_DIR = f"{BASE_DIR}/working/submission"
SUBMISSION_ZIP = f"{BASE_DIR}/working/submission.zip"
TASKS_PATH = f"{BASE_DIR}/input/google-code-golf-2025"

SUBMISSIONS_PATHS = [
    "/kaggle/input/solved-127-problems-local-pleaseupvote-submission/submission",
    "/kaggle/input/neurips-2025-google-code-golf-championship1",
    "/kaggle/input/liah-submission-gcgc-v2",
    "/kaggle/input/code-golf-ensemble-local-score-391-400-dsl",
]

# ===========================
# Проверка решения
# ===========================
def check_solution(solution_code, task_data):
    try:
        namespace = {}
        exec(solution_code, namespace)
        if 'p' not in namespace:
            return False
        examples = task_data['train'] + task_data['test'] + task_data['arc-gen']
        for example in examples:
            input_grid = copy.deepcopy(example['input'])
            expected = example['output']
            try:
                result = namespace['p'](input_grid)
                if result != expected:
                    return False
            except Exception:
                return False
        return True
    except Exception:
        return False

# ===========================
# Заглушки
# ===========================
def dummy_generate_solution(task_data):
    return "def p(g): return g"

def dummy_verify_solution(solution_code, task_data):
    return True

def generate_fallback(task_data):
    return "def p(g): return g"

# ===========================
# Обработка одной задачи
# ===========================
def process_task(task_num, predefined_solutions, arc_generator, generate_fallback_fn):
    task_id = f"{task_num:03d}"
    task_path = f"{TASKS_PATH}/task{task_id}.json"
    with open(task_path, 'r') as f:
        task_data = json.load(f)

    solutions = []
    solved_flag = False

    # 1. Проверка предопределенных решений
    if task_num in predefined_solutions:
        code = predefined_solutions[task_num]
        if check_solution(code, task_data):
            solutions.append(code)
            print_rich(f"[yellow]{task_id} - решено предопределенным решением[/yellow]")
            solved_flag = True

    # 2. Проверка существующих сабмишнов
    if not solved_flag:
        for sub_path in SUBMISSIONS_PATHS:
            code_path = f"{sub_path}/task{task_id}.py"
            if os.path.exists(code_path):
                with open(code_path, 'r') as f:
                    code = f.read()
                if check_solution(code, task_data):
                    solutions.append(code)
                    solved_flag = True
        if solved_flag:
            print_rich(f"[green]{task_id} - решено существующими сабмишнами[/green]")

    # 3. Генерация через AI
    if not solved_flag:
        try:
            generated_code = arc_generator['generate_solution'](task_data)
            if arc_generator['verify_solution'](generated_code, task_data):
                solutions.append(generated_code)
                print_rich(f"[yellow]{task_id} - решено генератором[/yellow]")
                solved_flag = True
        except Exception as e:
            print_rich(f"[red]Ошибка генерации решения для {task_id}: {e}[/red]")

    # 4. Попытка решить предопределенными решениями, если ничего не помогло
    if not solutions:
        for key, code in predefined_solutions.items():
            if check_solution(code, task_data):
                solutions.append(code)
                print_rich(f"[magenta]{task_id} - решено глобальным предопределенным решением (задача {key})[/magenta]")
                break

    # 5. Общий fallback
    if not solutions:
        fallback_code = generate_fallback_fn(task_data)
        print_rich(f"[red]{task_id} - решение fallback'ом[/red]")
        if fallback_code and check_solution(fallback_code, task_data):
            solutions.append(fallback_code)
            solved_flag = True

    # 6. Минимизация решений
    def get_bytes(code_str):
        return len(code_str.encode('utf-8'))

    def remove_spaces(code): return ''.join(code.split())
    def minimize_indentation(code): return '\n'.join(line.strip() for line in code.splitlines())
    def substitute_enumerate(code): return code.replace('enumerate', 'e')
    def substitute_range(code): return code.replace('range', 'r')
    def join_block_lines(code): return ''.join(line for line in code.splitlines() if line.strip())
    def strip_trailing_whitespaces(code): return '\n'.join(line.rstrip() for line in code.splitlines())
    def remove_empty_lines(code): return '\n'.join(line for line in code.splitlines() if line.strip())
    def remove_comments(code): return re.sub(r'#.*', '', code)
    # Можно оставить без изменений или добавить новые методы по желанию
    def shorten_variable_names(code): return code
    def def_to_lambda(code): return code

    methods = [
        remove_spaces, minimize_indentation, substitute_enumerate,
        substitute_range, join_block_lines, strip_trailing_whitespaces,
        remove_empty_lines, remove_comments
    ]

    best_solution = "def p(g): return g"
    total_score = 0.001

    if solutions:
        minimized_solutions = []
        for sol in solutions:
            for method in methods:
                try:
                    before_bytes = get_bytes(sol)
                    sol = method(sol)
                    after_bytes = get_bytes(sol)
                    if check_solution(sol, task_data) and after_bytes < before_bytes:
                        # Обновляем решение, если нашли более короткое
                        pass
                except Exception:
                    pass
            minimized_solutions.append(sol)
        best_solution = min(minimized_solutions, key=get_bytes)
        total_score = len(best_solution) / 1000
    else:
        # Если решений нет, используем дефолт
        best_solution = "def p(g): return g"
        total_score = len(best_solution) / 1000

    # Создаём файл решения
    os.makedirs(SUBMISSION_DIR, exist_ok=True)
    filepath = f"{SUBMISSION_DIR}/task{task_id}.py"
    with open(filepath, 'w') as f:
        f.write(best_solution)

    return total_score

# ===========================
# Основная функция
# ===========================
def main():
    # Предварительные решения (пока пусто)
    predefined_solutions = {}

    # Объект генератора решений
    arc_generator = {
        'generate_solution': dummy_generate_solution,
        'verify_solution': dummy_verify_solution
    }

    # Функция fallback
    generate_fallback_fn = generate_fallback

    total_solved = 0
    total_score = 0

    # Обрабатываем все задачи
    for task_num in tqdm(range(1, 401)):
        score = process_task(task_num, predefined_solutions, arc_generator, generate_fallback_fn)
        total_score += score
        if score > 0.001:
            total_solved += 1

    # Создаём zip-архив из подготовленных файлов
    with zipfile.ZipFile(SUBMISSION_ZIP, "w") as archive:
        for task_num in range(1, 401):
            task_id = f"{task_num:03d}"
            file_path = f"{SUBMISSION_DIR}/task{task_id}.py"
            if os.path.exists(file_path):
                archive.write(file_path, arcname=f"task{task_id}.py")
            else:
                print_rich(f"[red]Файл {file_path} не найден, пропускаю[/red]")

    print_rich(f"[green]Решено задач: {total_solved} / 400[/green]")
    print_rich(f"[blue]Общий скор: {total_score:.3f}[/blue]")

if __name__ == "__main__":
    main()

#  <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:black; font-size:120%; text-align:left;padding:3.0px; background: #cceeff; border-bottom: 8px solid #004466" > Сode golf<br><div>  
* [Constants and paths](#1)
* [Checking the solution](#2)
* [Stubs](#3)
* [Processing a single task](#4) 
* [Main function](#5) 
* [Сonclusion](#6)  
 

<a id="1"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#ffffff; font-size:120%; text-align:left;padding:3.0px; background: #003380; border-bottom: 10px solid #80ffff"> Constants and paths<br><div> 

In [ ]:
import os
import json
import zipfile
import copy
import re
from tqdm import tqdm
from rich import print as print_rich
import warnings

# ===========================
# Отключение предупреждений
# ===========================
warnings.filterwarnings("ignore", category=SyntaxWarning)

# ===========================
# Пути и константы
# ===========================
BASE_DIR = "/kaggle"
SUBMISSION_DIR = f"{BASE_DIR}/working/submission"
SUBMISSION_ZIP = f"{BASE_DIR}/working/submission.zip"
TASKS_PATH = f"{BASE_DIR}/input/google-code-golf-2025"

SUBMISSIONS_PATHS = [
    "/kaggle/input/solved-127-problems-local-pleaseupvote-submission/submission",
    "/kaggle/input/neurips-2025-google-code-golf-championship1",
    "/kaggle/input/liah-submission-gcgc-v2",
    "/kaggle/input/code-golf-ensemble-local-score-391-400-dsl",
]

<a id="2"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#ffffff; font-size: 90%; text-align:left;padding:4.0px; background: maroon; border-bottom: 5px solid black"># Checking the solution<br><div> 

In [ ]:
# ===========================
# Проверка решения
# ===========================
def check_solution(solution_code, task_data):
    try:
        namespace = {}
        exec(solution_code, namespace)
        if 'p' not in namespace:
            return False
        examples = task_data['train'] + task_data['test'] + task_data['arc-gen']
        for example in examples:
            input_grid = copy.deepcopy(example['input'])
            expected = example['output']
            try:
                result = namespace['p'](input_grid)
                if result != expected:
                    return False
            except Exception:
                return False
        return True
    except Exception:
        return False


<a id=""></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#ffffff; font-size: 90%; text-align:left;padding:4.0px; background: maroon; border-bottom: 5px solid black"># Stubs<br><div> 

In [ ]:
# ===========================
# Заглушки
# ===========================
def dummy_generate_solution(task_data):
    return "def p(g): return g"

def dummy_verify_solution(solution_code, task_data):
    return True

def generate_fallback(task_data):
    return "def p(g): return g"


<a id="4"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#ffffff; font-size:120%; text-align:left;padding:3.0px; background: #003380; border-bottom: 10px solid #80ffff"># Processing a single task<br><div> 

In [ ]:
# ===========================
# Обработка одной задачи
# ===========================
def process_task(task_num, predefined_solutions, arc_generator, generate_fallback_fn):
    task_id = f"{task_num:03d}"
    task_path = f"{TASKS_PATH}/task{task_id}.json"
    with open(task_path, 'r') as f:
        task_data = json.load(f)

    solutions = []
    solved_flag = False
        # 1. Проверка предопределенных решений
    if task_num in predefined_solutions:
        code = predefined_solutions[task_num]
        if check_solution(code, task_data):
            solutions.append(code)
            print_rich(f"[yellow]{task_id} - решено предопределенным решением[/yellow]")
            solved_flag = True

    # 2. Проверка существующих сабмишнов
    if not solved_flag:
        for sub_path in SUBMISSIONS_PATHS:
            code_path = f"{sub_path}/task{task_id}.py"
            if os.path.exists(code_path):
                with open(code_path, 'r') as f:
                    code = f.read()
                if check_solution(code, task_data):
                    solutions.append(code)
                    solved_flag = True
        if solved_flag:
            print_rich(f"[green]{task_id} - решено существующими сабмишнами[/green]")

    # 3. Генерация через AI
    if not solved_flag:
        try:
            generated_code = arc_generator['generate_solution'](task_data)
            if arc_generator['verify_solution'](generated_code, task_data):
                solutions.append(generated_code)
                print_rich(f"[yellow]{task_id} - решено генератором[/yellow]")
                solved_flag = True
        except Exception as e:
            print_rich(f"[red]Ошибка генерации решения для {task_id}: {e}[/red]")

    # 4. Попытка решить предопределенными решениями, если ничего не помогло
    if not solutions:
        for key, code in predefined_solutions.items():
            if check_solution(code, task_data):
                solutions.append(code)
                print_rich(f"[magenta]{task_id} - решено глобальным предопределенным решением (задача {key})[/magenta]")
                break

    # 5. Общий fallback
if not solutions:
    fallback_code = generate_fallback_fn(task_data)
    print_rich(f"[red]{task_id} - решение fallback'ом[/red]")
    if fallback_code and check_solution(fallback_code, task_data):
        solutions.append(fallback_code)
        solved_flag = True
    # 6. Минимизация решений
    def get_bytes(code_str):
        return len(code_str.encode('utf-8'))
            def remove_spaces(code): return ''.join(code.split())
    def minimize_indentation(code): return '\n'.join(line.strip() for line in code.splitlines())
    def substitute_enumerate(code): return code.replace('enumerate', 'e')
    def substitute_range(code): return code.replace('range', 'r')
    def join_block_lines(code): return ''.join(line for line in code.splitlines() if line.strip())
    def strip_trailing_whitespaces(code): return '\n'.join(line.rstrip() for line in code.splitlines())
    def remove_empty_lines(code): return '\n'.join(line for line in code.splitlines() if line.strip())
    def remove_comments(code): return re.sub(r'#.*', '', code)
    # Оставим без изменений
    def shorten_variable_names(code): return code
    def def_to_lambda(code): return code
            methods = [
        remove_spaces, minimize_indentation, substitute_enumerate,
        substitute_range, join_block_lines, strip_trailing_whitespaces,
        remove_empty_lines, remove_comments
    ]

    best_solution = "def p(g): return g"
    total_score = 0.001

    if solutions:
        minimized_solutions = []
        for sol in solutions:
            for method in methods:
                try:
                    before_bytes = get_bytes(sol)
                    sol = method(sol)
                    after_bytes = get_bytes(sol)
                    if check_solution(sol, task_data) and after_bytes < before_bytes:
                        # Обновляем решение, если нашли более короткое
                        pass
                except Exception:
                    pass
            minimized_solutions.append(sol)
        best_solution = min(minimized_solutions, key=get_bytes)
        total_score = len(best_solution) / 1000
    else:
        # Если решений нет, используем дефолт
        best_solution = "def p(g): return g"
        total_score = len(best_solution) / 1000

    # Создаём файл решения
    os.makedirs(SUBMISSION_DIR, exist_ok=True)
    filepath = f"{SUBMISSION_DIR}/task{task_id}.py"
    with open(filepath, 'w') as f:
        f.write(best_solution)

    return total_score


<a id="5"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#ffffff; font-size:120%; text-align:left;padding:3.0px; background: #003380; border-bottom: 10px solid #80ffff"> # Main function<br><div> 

In [ ]:
# ===========================
# Основная функция
# ===========================
def main():
    # Предварительные решения (пока пусто)
    predefined_solutions = {}

    # Объект генератора решений
    arc_generator = {
        'generate_solution': dummy_generate_solution,
        'verify_solution': dummy_verify_solution
    }

    # Функция fallback
    generate_fallback_fn = generate_fallback

    total_solved = 0
    total_score = 0

    # Обрабатываем все задачи
    for task_num in tqdm(range(1, 401)):
        score = process_task(task_num, predefined_solutions, arc_generator, generate_fallback_fn)
        total_score += score
        if score > 0.001:
            total_solved += 1

    # Создаём zip-архив из подготовленных файлов
    with zipfile.ZipFile(SUBMISSION_ZIP, "w") as archive:
        for task_num in range(1, 401):
            task_id = f"{task_num:03d}"
            file_path = f"{SUBMISSION_DIR}/task{task_id}.py"
            if os.path.exists(file_path):

<a id="6"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#ffffff; font-size:120%; text-align:left;padding:3.0px; background: #003380; border-bottom: 10px solid #80ffff"> # Сonclusion<br><div> 

* After processing each task, a taskXXX.py file is created inside SUBMISSION_DIR.
* All files are collected in a ZIP archive after the job is completed.
* There are stubs in the code for generating and checking solutions, which can be replaced or expanded.
* The entire process, from data loading, solution checking, generation, and minimization, is automated.